In [48]:
from ctypes import *
import imageio
import numpy as np
import matplotlib.pyplot as plt
import os
import imageio
import warnings
import time

In [49]:
cdll.LoadLibrary("Blink_SDK_C.dll")
slm_lib = CDLL("Blink_SDK_C")

In [50]:
LUT = 'slm_h2_encrypt_noPhaseWrap.txt'
blank_image = "512white.bmp"

LUT = bytes(LUT, 'utf-8')
lut_file = c_char_p(LUT)

create_SDK_func = slm_lib.Create_SDK
create_SDK_func.argtypes = (c_uint, c_uint, POINTER(c_uint), POINTER(c_bool), c_bool, c_bool, c_bool, c_uint, c_char_p)
# in python 3 the integer returned by create_SDK_func is bigger than 32 bit, so had to reassign the integer type to longlong
create_SDK_func.restype = c_longlong

# Basic parameters for calling Create_SDK
slm_resolution = c_uint(512)
# currently only supports 8 bit depth
bit_depth = c_uint(8)
num_boards_found = c_uint(0)
constructed_okay = c_bool(0)
is_nematic_type = c_bool(1)
RAM_write_enable = c_bool(1)
use_GPU = c_bool(1)
max_transients = c_uint(20)
true_frames = c_int(3)

# Call the Create_SDK constructor
# Returns a handle that's passed to subsequent SDK calls
sdk = create_SDK_func(bit_depth, slm_resolution, byref(num_boards_found), 
                     byref(constructed_okay), is_nematic_type, 
                     RAM_write_enable, use_GPU, max_transients,lut_file)


In [51]:
test_mask = np.ones((512,512), dtype= 'uint8')
n_bytes = c_uint(100)

In [65]:
#bool Calculate_transient_frames(Blink_SDK *sdk, const unsigned char* target_phase, unsigned int* byte_count);

Calculate_transient_frames_func = slm_lib.Calculate_transient_frames
Calculate_transient_frames_func.argtypes = (c_longlong, POINTER(c_ubyte), POINTER(c_uint))
Calculate_transient_frames_func.restype = c_bool

Retrieve_transient_frames_func = slm_lib.Retrieve_transient_frames
Retrieve_transient_frames_func.argtypes = (c_longlong, POINTER(c_ubyte))
Retrieve_transient_frames_func.restype = c_bool

Write_transient_frames_func = slm_lib.Write_transient_frames
Write_transient_frames_func.argtypes = (c_longlong, c_int, POINTER(c_ubyte), c_bool, c_bool)
Write_transient_frames_func.restype = c_bool
ctf_result = Calculate_transient_frames_func(sdk,test_mask.ctypes.data_as(POINTER(c_ubyte)), byref(n_bytes))
ctf_result

True

In [53]:
# get all tiffs in a folder
f_path = r'C:\Users\User\Documents\Code\SLM\Test masks'
warnings.filterwarnings('ignore')

tiff_list = [os.path.join(f_path,file) for file in os.listdir(f_path) if file.endswith('.tiff') or file.endswith('.tif')]
mask_list = [imageio.imread(tiff) for tiff in tiff_list]


In [68]:
#Precalculate_then_triggered_write function starts here

# fill first phase mask with random values
initial_phase = np.random.rand(512,512)
#where in memory the phase mask is
initial_pointer = initial_phase.ctypes.data_as(POINTER(c_ubyte))
n_bytes = c_uint(0)
#n_bytes should maybe be byref
okay = Calculate_transient_frames_func(sdk, initial_pointer, n_bytes)

if okay:
    okay = Retrieve_transient_frames_func(sdk, initial_pointer)
    
    
    
    
okay = Write_transient_frames_func(sdk,1, initial_pointer,c_bool(0), c_bool(0));

okay

True

In [ ]:
Delete_SDK_func = slm_lib.Delete_SDK
#for some reason giving this function an extra argument makes it work
Delete_SDK_func.argtypes = (c_longlong, c_bool)
SLM_power_func =  slm_lib.SLM_power
SLM_power_func.argtypes = (c_longlong, c_bool)




# Always call Delete_SDK before exiting
Delete_SDK_func(sdk, c_bool(1))

# Turn the SLM power off
SLM_power_func(sdk, c_bool(0))

In [ ]:
Is_slm_transient_constructed_func = slm_lib.Is_slm_transient_constructed
# Is_slm_transient_constructed_func.argtypes = (c_longlong, c_bool)
# Is_slm_transient_constructed_func.restype = c_bool


# Is_slm_transient_constructed_func(sdk, c_bool(1))